In [9]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
from langchain.agents.agent_toolkits import NLAToolkit
from langchain.tools.plugin import AIPlugin
import re
from dotenv import load_dotenv
import os
import json
import re
from langchain import Anthropic
from langchain.chat_models import ChatAnthropic
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    HumanMessage,
)
from typing import List
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.agents import tool
from pydantic import BaseModel, Field
from solcx import install_solc, compile_source
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, BaseMessage, HumanMessage, AIMessage
from dotenv import load_dotenv
from typing import List
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, BaseMessage, HumanMessage, AIMessage
from utils.tools import compile_solidity, SolidityCompilerInput


load_dotenv()


def get_secret(name):
    if name in os.environ:
        return os.environ[name]
    return os.getenv(name)

## Setup LLM

In [10]:
llm = OpenAI(temperature=0)

In [11]:
@tool(return_direct=False)
def compile_solidity(solidity_code_str):
    """
    This function compiles Solidity files using a specific version of solc.
    :return: Dict, the compiled contracts.
    """
    solidity_code_str = solidity_code_str.strip('`')
    solc_version = '0.8.2'
    # Install the specified version of solc
    install_solc(solc_version)
    try:
        result = compile_source(solidity_code_str)
    except Exception as e:
        print(f"couldn't compile {solidity_code_str}\n{str(e)}")
        return f'Compilation Failed. Either the code is invalid or current compiler({solc_version}) didnt match'
    print(result)
    return 'Compilation Successful. It is a valid solidity code.'

## Tool Retriever

We will use a vectorstore to create embeddings for each tool description. Then, for an incoming query we can create embeddings for that query and do a similarity search for relevant tools.

In [12]:
tools = [
    Tool(
        func=compile_solidity,
        name="solidity compiler",
        description="Can compile solidity code. Accepts ONLY solidity code",
        args_schema=SolidityCompilerInput,
    ),
]

We can now test this retriever to see if it seems to work.

## Prompt Template

The prompt template is pretty standard, because we're not actually changing that much logic in the actual prompt template, but rather we are just changing how retrieval is done.

In [13]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""

The custom prompt template now has the concept of a tools_getter, which we call on the input to select the tools to use

In [14]:
from typing import Callable
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    ############## NEW ######################
    # The list of tools available
    tools_getter: Callable
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        tools = self.tools_getter(kwargs["input"])
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

In [15]:
prompt = CustomPromptTemplate(
    template=template,
    tools_getter=tools,
    input_variables=["input", "intermediate_steps"]
)

ValidationError: 1 validation error for CustomPromptTemplate
tools_getter
  [Tool(name='solidity compiler', description='Can compile solidity code. Accepts ONLY solidity code', args_schema=<class 'utils.tools.SolidityCompilerInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, func=StructuredTool(name='compile_solidity', description='compile_solidity(solidity_code_str) - This function compiles Solidity files using a specific version of solc.\n    :return: Dict, the compiled contracts.', args_schema=<class 'pydantic.main.compile_soliditySchemaSchema'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, func=<function compile_solidity at 0x7f7d631e1240>, coroutine=None), coroutine=None)] is not callable (type=type_error.callable; value=[Tool(name='solidity compiler', description='Can compile solidity code. Accepts ONLY solidity code', args_schema=<class 'utils.tools.SolidityCompilerIn

## Output Parser

The output parser is unchanged from the previous notebook, since we are not changing anything about the output format.

In [ ]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [ ]:
output_parser = CustomOutputParser()

## Set up LLM, stop sequence, and the agent

Also the same as the previous notebook

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

## Use the Agent

Now we can use it!

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.run(solidity_code)